## HW5 Band Gap Prediction
### Darian Yang

You can start with basic models, and then try your best to optimize your predictions by using more sophisticated models, feature engineering, and fine-tuning the hyperparameters. The grade of this homework will be based on the score you get on Kaggle. 

There is no specific requirement for a written summary for this homework, but please leave some necessary notes/comments in your submission to help the grader understand your workflow.

#### TODO:
* Given the training data: X_train and Y_train
* Find best suitable features
* You are allowed to build any type of regression model
* You are allowed to use any type of data processing
* Find the best performing model
* Use your model to score Y_test

This dataset provides quantitative measurements of the band gap (Egap) for a set of inorganic crystaline materials.

#### File descriptions
* X_train_kaggle.csv - the training set: file with Material column
* y_train_kaggle.csv - the training set: Egap for training with Id column
* X_test_kaggle.csv - the test set: file with Material column with Id column that you should predict
* y_sample_submission.csv - a sample submission file in the correct format

#### Data fields
* Id - an id unique to a given material ( Please note 'Id' is the last column)
* D1-D132 - chemical descriptor to a given material
* Egap - Egap values in y- files

The evaluation metric for this competition is Mean absolute error .

#### Submission Format
For every molecule in the dataset, submission files should contain two columns: Id and Egap.

The file should contain a header and have the following format:
```
Id,Egap
1,0.456
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm

In [3]:
X_train = pd.read_csv("X_train_kaggle.csv", index_col=133).to_numpy()
y_train = pd.read_csv("y_train_kaggle.csv", index_col=1).to_numpy()
X_test = pd.read_csv("X_test_kaggle.csv", index_col=133).to_numpy()

In [4]:
TEST_ID = pd.read_csv("X_test_kaggle.csv",index_col=133).index

In [5]:
TEST_ID.shape

(4132,)

In [6]:
np.unique(TEST_ID).shape

(4132,)

In [7]:
TRAIN_ID = pd.read_csv("X_train_kaggle.csv",index_col=0).index

In [8]:
X_train.shape

(9640, 133)

Drop the material column:

In [9]:
X_train = X_train[:,1:]
X_train.shape

(9640, 132)

In [10]:
X_test = X_test[:,1:]
X_test.shape

(4132, 132)

In [11]:
y_train.shape

(9640, 1)

In [12]:
# reshape to 1d
y_train = y_train[:,0]
y_train.shape

(9640,)

In [13]:
def worker(model, save_to=None, X_train=X_train, y_train=y_train, X_test=X_test):
    
    model = model.fit(X_train, y_train)
    y_test = model.predict(X_test)

    if save_to:
        ret = {"Id":TEST_ID, "Egap":y_test}
        ret = pd.DataFrame(data=ret)
        ret.set_index("Id")
        ret.to_csv(save_to, index=False)
    
    return y_test

In [14]:
from sklearn.metrics import mean_squared_error

def calc_score(model, X_train, y_train, X_test, y_test):
    """
    Find the mse using an sklearn model.
    """
    model = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    # RF
    if hasattr(model, "oob_score_"):
        #print(f"OOB: {model.oob_score_}")
        return model.oob_score_, mse
    else:
        return mse

In [15]:
# first scale
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
# split training/test
X_train_b, X_test_b, y_train_b, y_test_b = \
    model_selection.train_test_split(X_train_scaled, y_train, test_size=0.6)

In [17]:
calc_score(linear_model.LinearRegression(), X_train_b, y_train_b, X_test_b, y_test_b)

1.1887630612528222

In [27]:
calc_score(RandomForestRegressor(max_depth=None, oob_score=True), X_train_b, y_train_b, X_test_b, y_test_b)

(0.6915955076438205, 0.7721188464802152)

In [218]:
calc_score(RandomForestRegressor(max_depth=None, oob_score=True, criterion="absolute_error"), X_train_b, y_train_b, X_test_b, y_test_b)

(0.6899596121795617, 0.768416162047233)

In [219]:
calc_score(RandomForestRegressor(max_depth=None, oob_score=True, criterion="poisson"), X_train_b, y_train_b, X_test_b, y_test_b)

(0.6913443127058705, 0.7730645766108283)

In [207]:
calc_score(svm.LinearSVR(), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

/ihome/lchong/dty7/Apps/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1.2211295173422594

For now, I will submit kaggle using random forest.

In [209]:
worker(RandomForestRegressor(max_depth=None), "rf.csv", X_train=X_train_scaled, y_train=y_train, X_test=X_test_scaled)

array([3.133309  , 2.57074427, 2.68455313, ..., 1.64507733, 1.623616  ,
       3.172862  ])

So between linear regression, a linear SVM, and random forest, RF is the best but note that we can't extrapolate with RF so it might not be the best option here. I will try optimizing SVMs since this dataset is smaller than the last HW.

In [210]:
calc_score(svm.LinearSVR(max_iter=10000, C=1), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

/ihome/lchong/dty7/Apps/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1.216609871226856

In [211]:
calc_score(svm.LinearSVR(max_iter=10000, C=10), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

/ihome/lchong/dty7/Apps/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1.2170656185325868

In [212]:
calc_score(svm.LinearSVR(max_iter=10000, C=100), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

/ihome/lchong/dty7/Apps/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


1.7660575937714196

In [213]:
calc_score(svm.SVR(kernel="linear", C=1, cache_size=1000), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

1.2166339185799946

In [214]:
calc_score(svm.SVR(kernel="rbf", C=1, cache_size=1000), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

0.9560429991532254

Note that from literature for a similar problem (https://pubs.acs.org/doi/full/10.1021/acs.jpclett.8b00124):

C and γ were optimized to 10 and 0.01 for SVR, respectively, while ϵ was set at 0.1

In [215]:
calc_score(svm.SVR(kernel="rbf", C=10, cache_size=1000), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

0.8657977947153491

In [216]:
calc_score(svm.SVR(kernel="rbf", C=10, gamma=0.01, cache_size=1000), X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

0.8647272137025013

In [217]:
calc_score(svm.SVR(kernel="rbf", C=10, gamma=0.01, epsilon=0.1, cache_size=1000), 
           X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

0.8647272137025013

Let's actually optimize this:

In [221]:
C_2d_range = [0.01, 0.1, 1, 10, 100]
gamma_2d_range = [0.01, 0.1, 1, 10, 100]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        clf = svm.SVR(C=C, gamma=gamma, cache_size=1000)
        clf.fit(X_train_b, y_train_b)
        y_pred = clf.predict(X_test_b)
        mse = mean_squared_error(y_test_b, y_pred)
        classifiers.append((C, gamma, clf, mse))

In [222]:
classifiers

[(0.01, 0.01, SVR(C=0.01, cache_size=1000, gamma=0.01), 1.8236485113967649),
 (0.01, 0.1, SVR(C=0.01, cache_size=1000, gamma=0.1), 2.5585353692385726),
 (0.01, 1, SVR(C=0.01, cache_size=1000, gamma=1), 2.602948806597704),
 (0.01, 10, SVR(C=0.01, cache_size=1000, gamma=10), 2.6043677219436296),
 (0.01, 100, SVR(C=0.01, cache_size=1000, gamma=100), 2.604429664100975),
 (0.1, 0.01, SVR(C=0.1, cache_size=1000, gamma=0.01), 1.2154735920005613),
 (0.1, 0.1, SVR(C=0.1, cache_size=1000, gamma=0.1), 2.196404116055674),
 (0.1, 1, SVR(C=0.1, cache_size=1000, gamma=1), 2.532059427595707),
 (0.1, 10, SVR(C=0.1, cache_size=1000, gamma=10), 2.5471858403151155),
 (0.1, 100, SVR(C=0.1, cache_size=1000, gamma=100), 2.5477969447261923),
 (1, 0.01, SVR(C=1, cache_size=1000, gamma=0.01), 0.9437444438182191),
 (1, 0.1, SVR(C=1, cache_size=1000, gamma=0.1), 1.3781565019410396),
 (1, 1, SVR(C=1, cache_size=1000, gamma=1), 2.2712499225498193),
 (1, 10, SVR(C=1, cache_size=1000, gamma=10), 2.363289627231711),
 

Interesting, the optimized rbf kernel svr is the same as from the paper, and overall probably not as good as RF.

I will submit to make sure:

In [223]:
worker(svm.SVR(kernel="rbf", C=10, gamma=0.01, cache_size=1000), "svm.csv", X_train=X_train_scaled, y_train=y_train, X_test=X_test_scaled)

array([3.69489438, 2.24564259, 3.23115338, ..., 1.64694377, 1.82499312,
       3.13586865])

As suspected, not as good as rf.

Okay, so let's optimize RF hyperparameters:

In [18]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [27]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=80, cv=3, verbose=1, n_jobs=8)

In [28]:
# Fit the random search model
rf_random.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 80 candidates, totalling 240 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 31.1min
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed: 110.5min
[Parallel(n_jobs=8)]: Done 240 out of 240 | elapsed: 127.2min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=80, n_jobs=8,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   verbose=1)

In [29]:
rf_random.best_params_

{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 20,
 'bootstrap': False}

In [30]:
calc_score(RandomForestRegressor(**rf_random.best_params_), 
           X_train=X_train_b, y_train=y_train_b, X_test=X_test_b, y_test=y_test_b)

0.7353321062429926

It does score better in mse compared to the default, let's submit it to kaggle.

In [31]:
worker(RandomForestRegressor(**rf_random.best_params_), "rf.csv", X_train=X_train_scaled, y_train=y_train, X_test=X_test_scaled)

array([3.14690964, 2.97176267, 2.64893846, ..., 1.80630918, 1.70350869,
       3.08582956])

It also scored well on kaggle, just barely above the optimized solution. So still a little more to go.

I'll try to narrow the RF paramater search using a more comprehensive gridsearchCV around the parameters identified from randomsearchCV.

In [32]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [30, 40, 50],
    'max_features': [11, 12],
    'min_samples_leaf': [1, 2],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [600, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=4, verbose=1)

In [ ]:
grid_search.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/home/dty7/Apps/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  4.1min


In [ ]:
grid_search.best_params_